In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/MyDrive/"
base_dir = root_dir + 'projects/'

Mounted at /content/gdrive


In [ ]:
!pip install tensorlayer

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/projects/HackathonCovid19Proj")
!pwd

from __future__ import print_function
import numpy.random as random
import torch.backends.cudnn as cudnn
import torch.nn as nn
from utils import *
from DataLoader import *
from Losses import *
import argparse
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from model.ResNet import resnet18


dic = {'CT-0': 0}
rev_dic = {0 : 'CT-0'}
metric_basic = ['acc']
metric_ad = ['MCC']
avg = ['micro', 'macro']
classes = len(dic)

def createTrainTestData(test_split=0.04, val_split=0.20, seed = 1997
              ):
    # Initialize
    if (not os.path.exists("./cache")):
        image_data, labels, cls_num_list = loadRawData()
        print(cls_num_list)
        image_data_per_class = [[] for i in range(len(image_data))]
        for i in range(classes):
            
            image_data_per_class[i] = concatenate(image_data[i])
            print("Shape for data samples in class " + rev_dic[i])
            print(image_data_per_class[i].shape)
            print(labels[i].shape)
            np.random.shuffle(image_data_per_class[i])

        # Split training set, validation set and test set
        X_train = image_data_per_class[0][:int(image_data_per_class[0].shape[0] * (1 - test_split))]
        Y_train = labels[0][:int(image_data_per_class[0].shape[0] * (1 - test_split))]
                          
        X_test = np.vstack( image_data_per_class[i][int(image_data_per_class[i].shape[0] * (1 - test_split)):]
                            for i in range(classes)
                             )
        Y_test= np.vstack( labels[i][int(image_data_per_class[i].shape[0] * (1 - test_split)):]
                            for i in range(classes)
                             )
        cls_num_list_np = np.array(cls_num_list)
        os.makedirs("./cache")
        np.save("./cache/X_train.npy", X_train)
        np.save("./cache/Y_train.npy", Y_train)
        #np.save("./cache64/X_val.npy", X_val)
        #np.save("./cache64/Y_val.npy", Y_val)
        np.save("./cache/X_test.npy", X_test)
        np.save("./cache/Y_test.npy", Y_test)
        np.save("./cache/cls.npy", cls_num_list_np)

    else:
        X_train = np.load("./cache/X_train.npy")
        Y_train = np.load("./cache/Y_train.npy")
        #X_val = np.load("./cache64/X_val.npy")
        #Y_val = np.load("./cache64/Y_val.npy")
        X_test = np.load("./cache/X_test.npy")
        Y_test = np.load("./cache/Y_test.npy")
        cls_num_list_np = np.load("./cache/cls.npy")
        cls_num_list = cls_num_list_np.tolist()

    state = np.random.get_state()
    np.random.shuffle(X_train)
    np.random.set_state(state)
    np.random.shuffle(Y_train)

    X_train = X_train[:, np.newaxis].astype(np.float32);
    #X_val = X_val[:, np.newaxis].astype(np.float32);
    X_test = X_test[:, np.newaxis].astype(np.float32);
    Y_train = np.squeeze(Y_train.astype(np.int64))
    #Y_val = np.squeeze(Y_val.astype(np.int64))
    Y_test = np.squeeze(Y_test.astype(np.int64))
    print("============Statistical Information===========")
    print("Training Set: ")
    print("Shape of data: " + str(X_train.shape) + "; Shape of labels: " + str(Y_train.shape))
    #print("Validation Set: ")
    #print("Shape of data: " + str(X_val.shape) + "; Shape of labels: " + str(Y_val.shape))
    print("Test Set: ")
    print("Shape of data: " + str(X_test.shape) + "; Shape of labels: " + str(Y_test.shape))
    # shuffle

    #return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test), cls_num_list[:classes]
    return (X_train, Y_train), (X_test, Y_test), cls_num_list[:classes]


# load data
print('Loading data...')
#train_set, val_set, test_set, cls_num_list = createTrainTestData()
train_set, test_set, cls_num_list = createTrainTestData()
X_train, Y_train = train_set
#X_val, Y_val = val_set
X_test, Y_test = test_set

/content/gdrive/MyDrive/projects/HackathonCovid19Proj
Loading data...
CT-0
Total number of images is 11134
There are 1 classes in total:
['CT-0']
Corresponding data sample number for each class:
[11134]
Image size:
(512, 512)
[11134]
Shape for data samples in class CT-0
(11134, 64, 64)
(11134, 1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


============Statistical Information===========
Training Set: 
Shape of data: (10688, 1, 64, 64); Shape of labels: (10688,)
Test Set: 
Shape of data: (446, 1, 64, 64); Shape of labels: (446,)


In [ ]:
#read data from cache

import os
os.chdir("/content/gdrive/MyDrive/projects/HackathonCovid19Proj")
!pwd

import numpy as np

X_train0 = np.load("./ct0/X_train.npy")
Y_train0 = np.load("./ct0/Y_train.npy")
X_test0 = np.load("./ct0/X_test.npy")
Y_test0 = np.load("./ct0/Y_test.npy")
X_train0 = X_train0[:, np.newaxis].astype(np.float32);
X_test0 = X_test0[:, np.newaxis].astype(np.float32);
Y_train0 = np.squeeze(Y_train0.astype(np.int64))
Y_test0 = np.squeeze(Y_test0.astype(np.int64))

X_train1 = np.load("./ct1/X_train.npy")
Y_train1 = np.load("./ct1/Y_train.npy")
X_test1 = np.load("./ct1/X_test.npy")
Y_test1 = np.load("./ct1/Y_test.npy")
X_train1 = X_train1[:, np.newaxis].astype(np.float32);
X_test1 = X_test1[:, np.newaxis].astype(np.float32);
Y_train1 = np.squeeze(Y_train1.astype(np.int64))
Y_test1 = np.squeeze(Y_test1.astype(np.int64))

X_train2 = np.load("./ct2/X_train.npy")
Y_train2 = np.load("./ct2/Y_train.npy")
X_test2 = np.load("./ct2/X_test.npy")
Y_test2 = np.load("./ct2/Y_test.npy")
X_train2 = X_train2[:, np.newaxis].astype(np.float32);
X_test2 = X_test2[:, np.newaxis].astype(np.float32);
Y_train2 = np.squeeze(Y_train2.astype(np.int64))
Y_test2 = np.squeeze(Y_test2.astype(np.int64))

X_train3 = np.load("./ct3/X_train.npy")
Y_train3 = np.load("./ct3/Y_train.npy")
X_test3 = np.load("./ct3/X_test.npy")
Y_test3 = np.load("./ct3/Y_test.npy")
X_train3 = X_train3[:, np.newaxis].astype(np.float32);
X_test3 = X_test3[:, np.newaxis].astype(np.float32);
Y_train3 = np.squeeze(Y_train3.astype(np.int64))
Y_test3 = np.squeeze(Y_test3.astype(np.int64))


print("============Statistical Information===========")
print("Training Set: ")
print("Shape of X_train0: " + str(X_train0.shape) + "; Shape of labels: " + str(Y_train0.shape))
print("Shape of X_train1: " + str(X_train1.shape) + "; Shape of labels: " + str(Y_train1.shape))
print("Shape of X_train2: " + str(X_train2.shape) + "; Shape of labels: " + str(Y_train2.shape))
print("Shape of X_train3: " + str(X_train3.shape) + "; Shape of labels: " + str(Y_train3.shape))

print("Test Set: ")
print("Shape of X_test0: " + str(X_test0.shape) + "; Shape of labels: " + str(Y_test0.shape))
print("Shape of X_test1: " + str(X_test1.shape) + "; Shape of labels: " + str(Y_test1.shape))
print("Shape of X_test2: " + str(X_test2.shape) + "; Shape of labels: " + str(Y_test2.shape))
print("Shape of X_test3: " + str(X_test3.shape) + "; Shape of labels: " + str(Y_test3.shape))


/content/gdrive/MyDrive/projects/HackathonCovid19Proj
============Statistical Information===========
Training Set: 
Shape of X_train0: (2448, 1, 64, 64); Shape of labels: (2448,)
Shape of X_train1: (6566, 1, 64, 64); Shape of labels: (6566,)
Shape of X_train2: (1200, 1, 64, 64); Shape of labels: (1200,)
Shape of X_train3: (432, 1, 64, 64); Shape of labels: (432,)
Test Set: 
Shape of X_test0: (102, 1, 64, 64); Shape of labels: (102,)
Shape of X_test1: (274, 1, 64, 64); Shape of labels: (274,)
Shape of X_test2: (50, 1, 64, 64); Shape of labels: (50,)
Shape of X_test3: (18, 1, 64, 64); Shape of labels: (18,)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from tqdm import tqdm
import anogan

X_trainmy = X_train0
print(X_trainmy.shape)

X_trainmy = X_trainmy.astype(np.float32)/255.
X_trainmy = np.transpose(X_trainmy, (0, 2, 3, 1))
print(X_trainmy.shape)

Model_d, Model_g = anogan.train(32, X_trainmy)

(2448, 1, 64, 64)
(2448, 64, 64, 1)
#### discriminator ######
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1088      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 64)        0         
                                                                 
 dropout (Dropout)           (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131200    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 128)       0         
                                                                 
 dropout_1 (Dropout)         (None, 16, 16, 128)       0         
                                                            

100%|██████████| 400/400 [39:12<00:00,  5.88s/it]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import anogan

#extra train

########################
#X_trainmy = X_train
#print(X_trainmy.shape)

#X_trainmy = X_trainmy.astype(np.float32)/255.
#X_trainmy = np.transpose(X_trainmy, (0, 2, 3, 1))
#print(X_trainmy.shape)
########################

def extra_train(BATCH_SIZE, X_train):
    d = anogan.discriminator_model()
    d.load_weights('assets/discriminator')
    g = anogan.generator_model()
    g.load_weights('assets/generator')

    d_on_g = anogan.generator_containing_discriminator(g, d)
    d.trainable = True
    for epoch in tqdm(range(100)):
        for index in range(int(X_train.shape[0]/BATCH_SIZE)):
            noise = np.random.uniform(0, 1, size=(BATCH_SIZE, 100))
            image_batch = X_train[index*BATCH_SIZE:(index+1)*BATCH_SIZE]
            generated_images = g.predict(noise, verbose=0)
            X = np.concatenate((image_batch, generated_images))
            y = np.array([1] * BATCH_SIZE + [0] * BATCH_SIZE)
            d_loss = d.train_on_batch(X, y)
            noise = np.random.uniform(0, 1, (BATCH_SIZE, 100))
            d.trainable = False
            g_loss = d_on_g.train_on_batch(noise, np.array([1] * BATCH_SIZE))
            d.trainable = True
        g.save_weights('assets/generator', True)
        d.save_weights('assets/discriminator', True)
    return d, g

extra_Model_d, extra_Model_g = extra_train(64, X_trainmy)


In [ ]:
import matplotlib.pyplot as plt
import anogan

## generate random image 

generated_img = anogan.generate(4)

for i in range(len(generated_img)):
    plt.figure(figsize=(2, 2))
    plt.imshow(generated_img[i].reshape(64, 64),cmap=plt.cm.gray)
    plt.show()

In [ ]:
!mkdir rst

# Append-adds at last
#file1 = open("./ano_score.txt", "a")  # append mode

myX_test = X_test0
myX_test = myX_test.astype(np.float32)/255.
myX_test = myX_test.reshape(-1, 64, 64, 1)
#X_train = X_train0
#X_train = X_train.astype(np.float32)/255.
#X_train = X_train.reshape(-1, 64, 64, 1)
y_test_label = 0

model = anogan.anomaly_detector()

for i in range(len(myX_test)):
    test_img = myX_test[i]
    #train_img = myX_train[i]
    testname = "./rst/" + str(y_test_label) + "_test" + str(i) + ".png"
    trainname = "./rst/train" + str(i) + ".png"
    fakename = "./rst/fake" + str(i) + ".png"
    rstname = "./rst/ano_test" + str(i) + ".png"
    
    ano_score, similar_img = anogan.compute_anomaly_score(model, test_img.reshape(1, 64, 64, 1))

    #plt.figure(figsize=(2, 2))
    #plt.imshow(train_img.reshape(64,64), cmap=plt.cm.gray)
    #plt.show()
    #plt.savefig(trainname)

    plt.figure(figsize=(2, 2))
    plt.imshow(test_img.reshape(64,64), cmap=plt.cm.gray)
    plt.show()
    #plt.savefig(testname)
    print("anomaly score : " + str(ano_score) + ", label :" + str(y_test_label) )
    #file1.write("anomaly score : " + str(ano_score) + ", label :" + str(y_test_label) + "\n")
    
    plt.figure(figsize=(2, 2))
    plt.imshow(similar_img.reshape(64,64), cmap=plt.cm.gray)
    plt.show()
    #plt.savefig(fakename)

    plt.figure(figsize=(2, 2))
    plt.imshow(test_img.reshape(64,64), cmap=plt.cm.gray)
    residual  = test_img.reshape(64,64) - similar_img.reshape(64, 64)
    plt.imshow(residual, cmap='jet', alpha=.5)
    plt.show()
    #plt.savefig(rstname)

#file1.close()    